# Exploration

A notebook used to explore FRED data

## Setup    

In [ ]:
# Load environment variables using dotenv

from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Setup autoreload of modules
import importlib


In [ ]:
from update_path import ensure_src_in_path

ensure_src_in_path()

In [ ]:
# Get the FRED API client

from fred_data import api

importlib.reload(api)

fred_client = api.FredApiClient()

In [ ]:
## Run everything above this cell for setup

## Data Exploration

### Releases

In [ ]:
# Fetch all current releases
from fred_data.releases import get_releases

releases = get_releases(fred_client)
releases_df = releases.releases.sort("name")

# Get an idea of the amount/types of data stored in releases
print(f"Release summary")
print(f"Count: {releases.count}")
print(f"Realtime range: {releases.realtime_start} to {releases.realtime_end}, Duration: {releases.realtime_end - releases.realtime_start}")
releases.releases.describe()

In [ ]:
# Fetch older and current releases

from datetime import date, timedelta
from fred_data.releases import get_releases

archived_releases = get_releases(fred_client, realtime_start=date.today() - timedelta(days=60), realtime_end=date.today())
archived_releases_df = archived_releases.releases

# Get an idea of the amount/types of data stored in releases
print(f"Archived Release summary")
print(f"Count: {archived_releases.count}")
print(f"Realtime range: {archived_releases.realtime_start} to {archived_releases.realtime_end}, Duration: {archived_releases.realtime_end - archived_releases.realtime_start}")
archived_releases.releases.describe()

### Release Dates

In [ ]:
# Get release dates for today and tomorrow
from datetime import date, timedelta

from fred_data.releases import get_release_dates

today = date.today()
tomorrow = today + timedelta(days=1)
release_dates = get_release_dates(fred_client, realtime_start=today, realtime_end=tomorrow, include_release_dates_with_no_data=True)
release_dates_df = release_dates.release_dates

print(f"Release Dates Summary")
print(f"Count: {release_dates.count}")
print(f"Realtime range: {release_dates.realtime_start} to {release_dates.realtime_end}, Duration: {release_dates.realtime_end - release_dates.realtime_start}")
release_dates_df.describe()

In [ ]:
# Get release dates from the past into the future
from datetime import date, timedelta

from fred_data.releases import get_release_dates

release_dates_over_time = get_release_dates(
    fred_client,
    realtime_start=date.today() - timedelta(days=30),
    realtime_end=date.today() + timedelta(days=10),
    include_release_dates_with_no_data=True
)
release_dates_over_time_df = release_dates_over_time.release_dates

print(f"Release Dates Summary")
print(f"Count: {release_dates_over_time.count}")
print(f"Realtime range: {release_dates_over_time.realtime_start} to {release_dates_over_time.realtime_end}, Duration: {release_dates_over_time.realtime_end - release_dates_over_time.realtime_start}")
release_dates_over_time_df.describe()

### Series

In [ ]:
# Get series search results from the API
import polars as pl

from fred_data.series import get_search_results

search_text = "illinois unemployment"

search_results = get_search_results(
    fred_client,
    search_text=search_text,
)
search_results_df = search_results.series

print(f"Search Results")
print(f"Count: {search_results.count}")
print(f"Realtime range: {search_results.realtime_start} to {search_results.realtime_end}, Duration: {search_results.realtime_end - search_results.realtime_start}")
search_results_df_summary = pl.DataFrame() if search_results_df.is_empty() else search_results_df.describe()
search_results_df_summary

In [ ]:
# Get all series contained within a given release

from fred_data.series import get_series_for_release


# Grabbed from releases_df from cells above
stl_financial_stress_release_id = 245
stl_financial_stress_multi_series = get_series_for_release(fred_client, stl_financial_stress_release_id)
stl_financial_stress_multi_series_df = stl_financial_stress_multi_series.series

print(f"Release Series Results")
print(f"Count: {stl_financial_stress_multi_series.count}")
print(f"Realtime range: {stl_financial_stress_multi_series.realtime_start} to {stl_financial_stress_multi_series.realtime_end}, Duration: {stl_financial_stress_multi_series.realtime_end - stl_financial_stress_multi_series.realtime_start}")
st_louis_financial_stress_series_summary = pl.DataFrame() if stl_financial_stress_multi_series_df.is_empty() else stl_financial_stress_multi_series_df.describe()
st_louis_financial_stress_series_summary
